In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time
import pickle

In [4]:
browser = webdriver.PhantomJS()

In [48]:
resort_urls = {'Loveland': 'colorado/loveland',
               'Arapahoe Basin': 'colorado/arapahoe-basin-ski-area',
               'Copper': 'colorado/copper-mountain-resort',
               'Eldora': 'colorado/eldora-mountain-resort',
               'Alpine Meadows': 'california/squaw-valley-usa',
               'Vail': 'colorado/vail',
               'Monarch': 'colorado/monarch-mountain',
               'Crested Butte': 'colorado/crested-butte-mountain-resort',
               'Taos': 'new-mexico/taos-ski-valley',
               'Diamond Peak': 'nevada/diamond-peak',
               'Winter Park': 'colorado/winter-park-resort',
               'Beaver Creek': 'colorado/beaver-creek'}

In [87]:
URL = 'http://www.onthesnow.com/'+resort_urls['Beaver Creek']+'/ski-resort.html'
browser.get(URL)
# time.sleep(3)

In [88]:
soup = BeautifulSoup(browser.page_source,'html.parser')
temp_rows = soup.select('p.bluetxt.temp') 
snowfall_rows = soup.select('p.bluetxt.sfa')
sdepth_rows = soup.select('p.bluetxt.sd') 

In [89]:
temp = int(''.join([x for x in [cell.text for cell in temp_rows][0] if x.isnumeric()]))
snowfall = int(''.join([x for x in [cell.text for cell in snowfall_rows][0] if x.isnumeric()]))
depth = int(''.join([x if x.isnumeric() else '0' for x in [cell.text for cell in sdepth_rows][0]]))

In [90]:
temp, snowfall, depth

(53, 0, 0)

In [91]:
def get_stats(resort):
    URL = 'http://www.onthesnow.com/'+resort_urls[resort]+'/ski-resort.html'
    browser.get(URL)
    soup = BeautifulSoup(browser.page_source,'html.parser')
    temp_rows = soup.select('p.bluetxt.temp') 
    snowfall_rows = soup.select('p.bluetxt.sfa')
    sdepth_rows = soup.select('p.bluetxt.sd') 
    temp = int(''.join([x for x in [cell.text for cell in temp_rows][0] if x.isnumeric()]))
    snowfall = int(''.join([x for x in [cell.text for cell in snowfall_rows][0] if x.isnumeric()]))
    depth = int(''.join([x if x.isnumeric() else '0' for x in [cell.text for cell in sdepth_rows][0]]))
    return temp, snowfall, depth

In [92]:
get_stats('Winter Park')

(53, 0, 0)

In [93]:
for resort in resort_urls:
    print(get_stats(resort))

(47, 0, 0)
(47, 0, 0)
(49, 0, 0)
(57, 0, 0)
(37, 0, 0)
(52, 0, 0)
(48, 0, 0)
(44, 0, 0)
(53, 0, 0)
(36, 0, 0)
(53, 0, 0)
(53, 0, 0)


In [140]:
stuff = soup.select('table.ovv_info tbody tr td')

In [141]:
rows = [cell.text for cell in stuff]
rows

['\nClosed\nSeason Start/End: 11/22 - 4/15\n',
 '8100ft - 11439ft',
 '\n19%|\n                42%|\n                22%|\n                17%\n',
 '25',
 '\n                N/A\n            ']

In [143]:
_, elevs, colors, lifts, price = rows

In [144]:
bottom = int(''.join([x for x in elevs.split()[0] if x.isnumeric()]))
top = int(''.join([x for x in elevs.split()[2] if x.isnumeric()]))
bottom, top

(8100, 11439)

In [121]:
greens = int(''.join([x for x in colors.split()[0] if x.isnumeric()]))
blues = int(''.join([x for x in colors.split()[1] if x.isnumeric()]))
blacks = int(''.join([x for x in colors.split()[2] if x.isnumeric()]))
bbs = int(''.join([x for x in colors.split()[3] if x.isnumeric()]))
greens, blues, blacks, bbs

(19, 42, 22, 17)

In [127]:
lifts = int(lifts)
lifts

25

In [139]:
price = int(''.join([x if x.isnumeric() else '0' for x in price.split()[0]]))
price

0

In [145]:
def get_elevs_colors_lifts_price(resort):
    URL = 'http://www.onthesnow.com/'+resort_urls[resort]+'/ski-resort.html'
    browser.get(URL)
    soup = BeautifulSoup(browser.page_source,'html.parser')
    stuff = soup.select('table.ovv_info tbody tr td')
    rows = [cell.text for cell in stuff]
    _, elevs, colors, lifts, price = rows
    bottom = int(''.join([x for x in elevs.split()[0] if x.isnumeric()]))
    top = int(''.join([x for x in elevs.split()[2] if x.isnumeric()]))
    greens = int(''.join([x for x in colors.split()[0] if x.isnumeric()]))
    blues = int(''.join([x for x in colors.split()[1] if x.isnumeric()]))
    blacks = int(''.join([x for x in colors.split()[2] if x.isnumeric()]))
    bbs = int(''.join([x for x in colors.split()[3] if x.isnumeric()]))
    lifts = int(lifts)
    price = int(''.join([x if x.isnumeric() else '0' for x in price.split()[0]]))
    return bottom, top, greens, blues, blacks, bbs, lifts, price

In [ ]:
elevs_colors_lifts_price = {}
for resort in resort_urls:
    elevs